In [ ]:
import numpy as np
np.set_printoptions(suppress=True) 

# Datasets
from aif360.datasets import MEPSDataset19
### fyi: there are also alternate MEPSDataset data sets to look into

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

## utility functions
from common_utils import compute_metrics ## taken from AIF360 github repo
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

## In-processing prejudice remover
from aif360.algorithms.inprocessing import PrejudiceRemover

## Adversarial debiasing
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
import tensorflow as tf

## Load a dataset regarding healthcare allocation
### p 136

In [ ]:
np.random.seed(132)
def split_data_trn_vld_tst(data_raw):
    dset_raw_trn, dset_raw_vt = data_raw.split([0.7], shuffle=True)
    dset_raw_vld, dset_raw_tst = dset_raw_vt.split([0.5], shuffle=True)
    
    return dset_raw_trn, dset_raw_vld, dset_raw_tst

In [ ]:
## p 137
med_data = MEPSDataset19()

In [ ]:
dset_raw_trn, dset_raw_vld, dset_raw_tst = split_data_trn_vld_tst(med_data)

In [ ]:
dset_raw_trn.protected_attribute_names

In [ ]:
priv_group   = [{'RACE': 1}]
unpriv_group = [{'RACE': 0}]

In [ ]:
dset_raw_trn

In [ ]:
dset_raw_trn.label_names

In [ ]:
metric = BinaryLabelDatasetMetric(dset_raw_trn, 
                                  unprivileged_groups = unpriv_group, 
                                  privileged_groups   = priv_group)

In [ ]:
metric.disparate_impact()

In [ ]:
metric.consistency()

In [ ]:
metric.mean_difference()

## Prejudice Remover

In [ ]:
## p 143
def test_eta_bal_acc(ETA, dset_raw_trn, dset_raw_vld, dset_raw_tst):
    pr = PrejudiceRemover(sensitive_attr = 'RACE', eta = ETA)
    scaler = StandardScaler()

    dset_scaled_trn = dset_raw_trn.copy()
    dset_scaled_trn.features = scaler.fit_transform(dset_scaled_trn.features)

    pr_fitted = pr.fit(dset_scaled_trn)
    
    accs = []
    thresholds = np.linspace(0.01, 0.50, 10)

    dset_val = dset_raw_vld.copy()
    dset_val.features = scaler.transform(dset_val.features)

    ##################### STEP 1 TRAINING WITH INPROCESSING #####################
    pr_pred_prob = pr_fitted.predict(dset_val).scores

    ##################### STEP 2 PICKING THRESHOLD WITH VALIDATION DATA #####################
    for threshold in thresholds:
        dset_val_pred = dset_val.copy()
        dset_val_pred.labels = (pr_pred_prob[:, 0] > threshold).astype(np.float64)

        metric = ClassificationMetric(
                    dset_val, dset_val_pred,
                    unprivileged_groups = unpriv_group,
                    privileged_groups=priv_group)
        accs.append((metric.true_positive_rate() + metric.true_negative_rate()) / 2)


    pr_val_best_idx = np.argmax(accs)
    best_threshold = thresholds[pr_val_best_idx]
    
    ##################### STEP 3 TEST DATA #####################
    dset_tst = dset_raw_tst.copy()
    dset_tst.features = scaler.transform(dset_tst.features)

    pr_pred_prob = pr_fitted.predict(dset_tst).scores


    dset_tst_pred = dset_tst.copy()
    dset_tst_pred.labels = (pr_pred_prob[:, 0] > best_threshold).astype(np.float64)

    metric = ClassificationMetric(
                dset_tst, dset_tst_pred,
                unprivileged_groups = unpriv_group,
                privileged_groups   = priv_group)
    test_acc = (metric.true_positive_rate() + metric.true_negative_rate()) / 2 ## no built in balanced error rate
    test_disp_impact = metric.disparate_impact()

    print("Testing accuracy with ETA %0.2f = %0.2f\n Disparate impact %0.2f" % (ETA, test_acc, test_disp_impact))
    return (test_acc, test_disp_impact)

In [ ]:
## p 144
test_eta_bal_acc(5.0, dset_raw_trn, dset_raw_vld, dset_raw_tst)

In [ ]:
test_eta_bal_acc(50.0, dset_raw_trn, dset_raw_vld, dset_raw_tst)

In [ ]:
test_eta_bal_acc(20.0, dset_raw_trn, dset_raw_vld, dset_raw_tst)

## Adversarial debiasing
### p 145

In [ ]:
## p 148
tf.reset_default_graph()
sess = tf.Session()
 
kwargs = {'privileged_groups'        : priv_group,
        'unprivileged_groups'        : unpriv_group,
        'scope_name'                 : 'debiased_classifier', 
        'debias'                     : True,
        'sess'                       : sess,
        'adversary_loss_weight'      : 0.5,
        'num_epochs'                 : 2, 
        'batch_size'                 : 128, 
        'classifier_num_hidden_units': 200, 
        'debias'                     : False,
        'seed'                       : 117
         }

# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(**kwargs) 

## p 149
scaler = StandardScaler()

dset_scaled_trn = dset_raw_trn.copy()
dset_scaled_trn.features = scaler.fit_transform(dset_scaled_trn.features)

debiased_model.fit(dset_scaled_trn)

dset_tst               = dset_raw_tst.copy()
dset_tst.features      = scaler.transform(dset_tst.features)

thresholds = np.linspace(0.2, 0.60, 5)

for thresh in thresholds:
    dset_tst_pred          = dset_tst.copy()
    dset_tst_pred.labels   = debiased_model.predict(dset_tst).scores > thresh
    print(np.bincount(dset_tst_pred.labels[:, 0].astype('int')))
    
    adv_deb_metric = ClassificationMetric(
                        dset_tst, dset_tst_pred,
                        unprivileged_groups = unpriv_group,
                        privileged_groups   = priv_group)

    test_acc = (adv_deb_metric.true_positive_rate() + adv_deb_metric.true_negative_rate()) / 2
    test_disp_impact = adv_deb_metric.disparate_impact()

    print("\n\nThresh: %0.2f\nTesting balanced accuracy %0.2f\nDisparate impact %0.2f" % 
          (thresh, test_acc, test_disp_impact))

In [ ]:
## p 150
tf.reset_default_graph()
sess = tf.Session()
 
kwargs = {
    'privileged_groups'        : priv_group,
    'unprivileged_groups'        : unpriv_group,
    'scope_name'                 : 'debiased_classifier', 
    'debias'                     : True,
    'sess'                       : sess,
    'adversary_loss_weight'      : 1.0,
    'num_epochs'                 : 25, 
    'batch_size'                 : 128, 
    'classifier_num_hidden_units': 16, 
    'debias'                     : True,
    'seed'                       : 117
     }

# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(**kwargs) 

scaler = StandardScaler()

dset_scaled_trn = dset_raw_trn.copy()
dset_scaled_trn.features = scaler.fit_transform(dset_scaled_trn.features)

debiased_model.fit(dset_scaled_trn)

dset_tst               = dset_raw_tst.copy()
dset_tst.features      = scaler.transform(dset_tst.features)

thresholds = np.linspace(0.2, 0.60, 5)

for thresh in thresholds:
    dset_tst_pred          = dset_tst.copy()
    dset_tst_pred.labels   = debiased_model.predict(dset_tst).scores > thresh
    print(np.bincount(dset_tst_pred.labels[:, 0].astype('int')))
    
    adv_deb_metric = ClassificationMetric(
                        dset_tst, dset_tst_pred,
                        unprivileged_groups = unpriv_group,
                        privileged_groups   = priv_group)

    test_acc = (adv_deb_metric.true_positive_rate() + adv_deb_metric.true_negative_rate()) / 2
    test_disp_impact = adv_deb_metric.disparate_impact()

    print("\n\nThresh: %0.2f\nTesting balanced accuracy %0.2f\nDisparate impact %0.2f" % (thresh, test_acc, test_disp_impact))